In [1]:
import rospy
import numpy as np
from cookiecutter.main import cookiecutter
import collections
from pathlib import Path
import json
import shutil

In [2]:
def get_param(name, default):
    try:
        value = rospy.get_param(name, default)
    except ConnectionRefusedError:
        value = default
    print('param "{}" = "{}"'.format(name, value))
    return value

SEED = int(get_param('~seed', '178'))
WORLD_NAME = get_param('~world_name', 'tomato_field')
MODEL_NAME_PREFIX = get_param('~model_name_prefix', 'tomato')
OUT_PATH = Path(get_param('~out_path', Path.cwd() / '../generated/test01')).resolve()
MODEL_TEMPLATE = Path(get_param('~model_template', Path.cwd() / '../templates/tomato_model')).resolve()
WORLD_TEMPLATE = Path(get_param('~world_template', Path.cwd() / '../templates/tomato_world')).resolve()
ROW_COUNT = int(get_param('~row_count', '3'))
ROW_LENGTH = int(get_param('~row_length', '6'))
ROW_DIST = float(get_param('~row_dist', '2.0'))
CROP_DIST = float(get_param('~crop_dist', '0.9'))

shutil.rmtree(OUT_PATH, ignore_errors=True)
np.random.seed(SEED)


param "~seed" = "178"
param "~world_name" = "tomato_field"
param "~model_name_prefix" = "tomato"
param "~out_path" = "/home/nicola/catkin_ws/src/fields-ignition/fields_ignition/scripts/../generated/test01"
param "~model_template" = "/home/nicola/catkin_ws/src/fields-ignition/fields_ignition/scripts/../templates/tomato_model"
param "~world_template" = "/home/nicola/catkin_ws/src/fields-ignition/fields_ignition/scripts/../templates/tomato_world"
param "~row_count" = "3"
param "~row_length" = "6"
param "~row_dist" = "2.0"
param "~crop_dist" = "0.9"


In [3]:
# helper class to build the markers.json
class Markers:
    markers = []
    last_id = 0

    @staticmethod
    def next_id():
        Markers.last_id += 1
        return Markers.last_id

    @staticmethod
    def reset():
        Markers.markers = []
    
    @staticmethod
    def add_plant(x, y, z):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'PLANT',
            'id': id,
            'translation': [x, y, z]
        })
        return id
    

    @staticmethod
    def add_fruit(x, y, z, plant_id):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'FRUIT',
            'id': id,
            'translation': [x, y, z],
            'plant_id': plant_id
        })
        return id
    
    @staticmethod
    def dumps():
        return json.dumps(Markers.markers, indent=4)

In [4]:
models = {'list': []}
Markers.reset()

for x in range(ROW_COUNT):
    for y in range(ROW_LENGTH):
        model_name = 'tomato_{}'.format(x * ROW_LENGTH + y)

        cookiecutter(str(MODEL_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'model_name': model_name})

        x_pos, y_pos, z_pos = x * ROW_DIST, y * CROP_DIST, 0
        models['list'].append({
            'model': model_name,
            'name': model_name,
            'pose': '{} {} 0 0 0 0'.format(x_pos, y_pos)
        })
        x_pos += np.random.uniform(-0.1, 0.1)
        y_pos += np.random.uniform(-0.1, 0.1)
        seed = np.random.randint(10000)
        dir = (OUT_PATH / WORLD_NAME / model_name).resolve()
        dir_blender = (Path.cwd() / '../blender').resolve()
        blend = str(dir_blender / 'tomato_gen.blend')
        script = str(dir_blender / 'tomato_gen.py')
        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
        with open(dir / 'markers.json') as markers_file:
            plant_markers = json.load(markers_file)
            for marker in plant_markers:
                if marker['marker_type'] == 'FRUIT':
                    Markers.add_fruit(
                        marker['translation'][0] + x_pos,
                        marker['translation'][1] + y_pos,
                        marker['translation'][2] + z_pos,
                        plant_id
                    )
            

cookiecutter(str(WORLD_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'models': models})

with open(OUT_PATH / WORLD_NAME / 'markers.json', 'w') as outfile:
    json.dump(Markers.markers, outfile, indent=4, sort_keys=True)

Blender 2.82 (sub 7)
Read blend: /home/nicola/catkin_ws/src/fields-ignition/fields_ignition/blender/tomato_gen.blend
Error: File written by newer Blender binary (290.0), expect loss of data!
Traceback (most recent call last):
  File "/home/nicola/catkin_ws/src/fields-ignition/fields_ignition/blender/tomato_gen.py", line 238, in <module>
    gen_plant(write_results=True)
  File "/home/nicola/catkin_ws/src/fields-ignition/fields_ignition/blender/tomato_gen.py", line 200, in gen_plant
    bpy.ops.object.parent_clear(
  File "/usr/share/blender/scripts/modules/bpy/ops.py", line 199, in __call__
    ret = op_call(self.idname_py(), C_dict, kw, C_exec, C_undo)
RuntimeError: Operator bpy.ops.object.parent_clear.poll() failed, context is incorrect

Blender quit


FileNotFoundError: [Errno 2] No such file or directory: '/home/nicola/catkin_ws/src/fields-ignition/fields_ignition/generated/test01/tomato_field/tomato_0/markers.json'